In [1]:
import re
import glob
import joblib
import feather
import numpy as np
import pandas as pd
from datetime import date
from currency_converter import CurrencyConverter

from src.clean_data import (
    clean_procedimientos_file,
    convert_to_mxn,
    get_claveuc_proc,
    get_claveuc_nombre,
    get_claveuc_real,
    convert_to_mxn
)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%config IPCompleter.use_jedi = False
%matplotlib inline

In [2]:
file_names = [
    '../data/raw/Procedimiento de Contratación 2010-2012.xlsx',
    '../data/raw/Procedimiento de Contratación 2013.xlsx',
    '../data/raw/Procedimiento de Contratación 2014.xlsx',
    '../data/raw/Procedimiento de Contratación 2015.xlsx',
    '../data/raw/Procedimiento de Contratación 2016.xlsx',
    '../data/raw/Procedimiento de Contratación 2017.xlsx'
]

converter_pesos = CurrencyConverter(fallback_on_missing_rate=True, verbose=False, fallback_on_wrong_date=True)

## Clean files

In [3]:
df_procedimientos = joblib.Parallel(n_jobs=-1, batch_size=1)(
    joblib.delayed(clean_procedimientos_file)(name) for name in file_names
)

df_procedimientos = pd.concat(df_procedimientos, axis=0, ignore_index=True)

# Just procedimientos between 2012 and 2017
df_procedimientos = df_procedimientos.loc[(df_procedimientos.FECHA_INICIO.dt.year > 2011), :]

ordered_cols = [
    'GOBIERNO',
    # Dependencia/unidad/responsable
    'DEPENDENCIA',
    'SIGLAS',    
    'NOMBRE_DE_LA_UC',
    'CLAVEUC',
    'RESPONSABLE',
    'ESTRATIFICACION_MUC',
    # proveedor
    'FOLIO_RUPC',
    'PROVEEDOR_CONTRATISTA',
    'ESTATUS_EMPRESA',
    'ESTRATIFICACION_MPC',
    'CLAVE_CARTERA_SHCP',
    # montos
    'IMPORTE_CONTRATO',
    'APORTACION_FEDERAL',
    'MONEDA',
    # datos procedimiento
    'NUMERO_PROCEDIMIENTO',
    'FORMA_PROCEDIMIENTO',
    'TIPO_PROCEDIMIENTO',
    # datos contrato
    'CODIGO_CONTRATO',
    'TITULO_CONTRATO',
    'IDENTIFICADOR_CM',
    'TIPO_CONTRATACION',
    'ESTATUS_CONTRATO',
    'COMPRA_CONSOLIDADA',
    'PLURIANUAL',
    'CARACTER',
    'CONTRATO_MARCO',
    'CONVENIO_MODIFICATORIO',

    # Fechas ordenadas
    'PROC_F_PUBLICACION',
    'FECHA_APERTURA_PROPOSICIONES',
    'EXP_F_FALLO',
    'FECHA_CELEBRACION',
    'FECHA_INICIO',
    'FECHA_FIN',
    # expediente
    'CODIGO_EXPEDIENTE',
    'TITULO_EXPEDIENTE',
    'PLANTILLA_EXPEDIENTE',
    # otros
    'CLAVE_PROGRAMA',
    'CUENTA_ADMINISTRADA_POR',
    'ANUNCIO',
    'ARCHIVADO',
    'SIGLAS_PAIS',
    'RAMO',
    'ORGANISMO',
    'C_EXTERNO',
]

df_procedimientos = df_procedimientos.loc[:, ordered_cols]
print(df_procedimientos.shape) # 861169

(888441, 45)


In [4]:
dep_repetidas = {
    'HOSPITAL GENERAL DE MEXICO "DR. EDUARDO LICEAGA"': 'HOSPITAL GENERAL DE MEXICO',
    'INSTITUTO NACIONAL DE REHABILITACION LUIS GUILLERMO IBARRA IBARRA': 'INSTITUTO NACIONAL DE REHABILITACION',
    'CENTRO DE INVESTIGACION EN GEOGRAFIA Y GEOMATICA, "ING. JORGE L. TAMAYO", A.C.': 'CENTRO DE INVESTIGACION EN GEOGRAFIA Y GEOMATICA "ING. JORGE L. TAMAYO", A.C.',
    'TRIBUNAL FEDERAL DE JUSTICIA FISCAL Y ADMINISTRATIVA': 'TRIBUNAL FEDERAL DE JUSTICIA ADMINISTRATIVA',
    'INSTITUTO DE INVESTIGACIONES ELECTRICAS': 'INSTITUTO NACIONAL DE ELECTRICIDAD Y ENERGIAS LIMPIAS'
}

for k, v in dep_repetidas.items():
    df_procedimientos.loc[df_procedimientos.DEPENDENCIA == k, 'DEPENDENCIA'] = v

del dep_repetidas

df_procedimientos = df_procedimientos.assign(
    CLAVEUC_PROC=df_procedimientos.NUMERO_PROCEDIMIENTO.map(get_claveuc_proc)
)
df_procedimientos = df_procedimientos.assign(
    CLAVEUC_NOM=df_procedimientos.NOMBRE_DE_LA_UC.map(get_claveuc_nombre)
)

# Clean CLAVEUC
df_procedimientos.loc[
    df_procedimientos.CLAVEUC.str.len() == 8, 'CLAVEUC'] = df_procedimientos.CLAVEUC.str.pad(9, fillchar='0')
df_procedimientos.loc[
    df_procedimientos.CLAVEUC.str.len() == 7, 'CLAVEUC'] = df_procedimientos.CLAVEUC.str.pad(9, fillchar='0')


# Read valid unidades compradoras
unidades_directorio = pd.read_excel(
    '../data/raw/Directorio Unidades Compradoras Gobierno Federal.xls', dtype=str)
unidades_directorio = set(
    unidades_directorio.loc[:, 'Clave UC en CompraNet'].unique())

unidades_est = pd.read_excel(
    '../data/raw/Directorio Unidades Compradoras Gobierno Estatal.xls', dtype=str)
unidades_est = set(
    unidades_est.loc[:, 'Clave UC en CompraNet'].unique())

unidades_mun = pd.read_excel(
    '../data/raw/Directorio Unidades Compradoras Gobierno Municipal.xls', dtype=str)
unidades_mun = set(
    unidades_mun.loc[:, 'Clave UC en CompraNet'].unique())

unidades_directorio = unidades_directorio | unidades_est | unidades_mun
print(len(unidades_directorio))

print(df_procedimientos.shape)

df_procedimientos = get_claveuc_real(df_procedimientos, unidades_directorio)

# Transform to mexican pesos
monto_en_pesos = convert_to_mxn(
    converter_pesos, df_procedimientos.IMPORTE_CONTRATO.values,
    df_procedimientos.MONEDA.values, df_procedimientos.FECHA_INICIO
)

df_procedimientos = df_procedimientos.assign(IMPORTE_PESOS=monto_en_pesos)

del monto_en_pesos
print(df_procedimientos.shape)

5181
(888441, 47)
(888441, 49)


## Guardar

In [8]:
df_procedimientos = df_procedimientos.drop(['CLAVE_CARTERA_SHCP', 'CLAVEUC_PROC', 'CLAVEUC_NOM'], axis=1)

feather.write_dataframe(
    df_procedimientos, '../data/processed/procedimientos_all_2017_09_25.feather')
df_procedimientos.to_csv(
    '../data/processed/procedimientos_all_2017_09_25.psv', sep='|', encoding='utf-8', index=False, quoting=1)

## Test saved DF

In [5]:
df_test = feather.read_dataframe('../data/processed/procedimientos_all_2017_09_25.feather')
print(df_test.shape)
df_test.head()

(888441, 46)


,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,CLAVEUC_REAL,IMPORTE_PESOS
0,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,RAUL EUGENIO ALVAREZ CARRASCO,HABILITADO,MICRO,512930.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,208993,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-12-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN,901024986,512930.40
1,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,MA DE LA LUZ TISCARENO ELIAS,HABILITADO,NO MIPYME,1887409.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,208992,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-10-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN,901024986,1887409.40
2,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,PAPELERIA CONSUMIBLES Y ACCESORIOS,HABILITADO,MICRO,1892535.82,NaN,MXN,IA-901024986-N53-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,257525,IEADJCONT 1140/2012,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-11-05 14:34:00,2012-11-12 10:00:00,2012-10-19,NaT,2012-11-12,2012-12-31,268049,IEA PAPELERIA Y CONSUMIBLES,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN,901024986,1892535.82
3,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS,HABILITADO,PEQUENA,24234.00,NaN,MXN,AA-901024986-N31-2012,None,ADJUDICACION DIRECTA FEDERAL,241054,CONSUMIBLES,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-09-06 08:49:00,2012-09-07 09:00:00,NaT,NaT,2012-09-11,2012-12-31,252725,COORDINACION ESTATAL DE ASESORIA Y SEGUIMIENTO...,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,None,No,MX,NaN,None,NaN,901024986,24234.00
4,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS,HABILITADO,PEQUENA,18700.00,NaN,MXN,AA-901024986-N64-2012,None,ADJUDICACION DIRECTA FEDERAL,474294,ADQUISICION DE EQUIPO DE COMPUTO,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-12-12 11:38:00,2012-12-14 10:00:00,NaT,NaT,2012-12-17,2012-12-31,320854,EQUIPO DE COMPUTO DESARROLLO EDUCATIVO,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,None,No,MX,NaN,None,NaN,901024986,18700.00


In [9]:
df_test.loc[df_test.CODIGO_EXPEDIENTE == '193814']

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,CLAVEUC_REAL,IMPORTE_PESOS
49374,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,CASTRO LOPEZ ANA MARGARITA,HABILITADO,None,50850.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,292232,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-04-20,2012-04-19,2012-04-26,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,50850.00
49375,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,3750.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293953,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-10-22,2012-10-21,2012-10-29,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,3750.00
49376,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,66000.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293944,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-09-18,2012-09-17,2012-09-24,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,66000.00
49377,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,90000.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293943,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-05-02,2012-05-01,2012-05-08,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,90000.00
49378,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,18000.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293923,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-05-15,2012-05-14,2012-05-21,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,18000.00
49379,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,21600.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293896,MATERIALESACCESORIOS Y SUMINISTRO

In [12]:
df_test.loc[df_test.NUMERO_PROCEDIMIENTO == 'AA-012NAW001-N49-2012']

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,CLAVEUC_REAL,IMPORTE_PESOS
49374,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,CASTRO LOPEZ ANA MARGARITA,HABILITADO,None,50850.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,292232,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-04-20,2012-04-19,2012-04-26,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,50850.00
49375,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,3750.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293953,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-10-22,2012-10-21,2012-10-29,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,3750.00
49376,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,66000.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293944,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-09-18,2012-09-17,2012-09-24,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,66000.00
49377,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,90000.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293943,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-05-02,2012-05-01,2012-05-08,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,90000.00
49378,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,18000.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293923,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-01-16,NaT,2012-05-15,2012-05-14,2012-05-21,193814,MATERIALESACCESORIOS Y SUMINISTROS MEDICOS,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,012NAW001,18000.00
49379,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,LOPEZ ROSALES MARIO SANTIAGO,HABILITADO,None,21600.00,NaN,MXN,AA-012NAW001-N49-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,293896,MATERIALESACCESORIOS Y SUMINISTRO